In [4]:
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import os, shutil, openpyxl, csv, time
import pandas as pd

In [5]:
download_folder = 'C:/MyProjects/WebScraping/Upwork/Ahmad/Job2/downloads'
options = uc.ChromeOptions()
options.add_argument('--disable-popup-blocking')  # Optional: Disable popup blocking
options.add_experimental_option("prefs", {
    "download.default_directory": download_folder,  # Set default download directory
    "download.prompt_for_download": False,  # Disable download prompt
    "directory_upgrade": True  # Ensure the folder is created if it doesn't exist
})
driver = uc.Chrome(options=options)
driver.maximize_window()

In [84]:
driver.get('https://training.gov.au/search?searchText=&searchType=RTO')
time.sleep(5)
driver.get('https://training.gov.au/search?searchText=&searchType=RTO&status=0&status=2&rtoTypeCode=31&rtoTypeCode=21&rtoTypeCode=25&rtoTypeCode=27&rtoTypeCode=61&rtoTypeCode=51&rtoTypeCode=53&rtoTypeCode=91&rtoTypeCode=93&rtoTypeCode=95&rtoTypeCode=97&rtoTypeCode=99&deliveredLocationState=Vic')


In [85]:
rtos = driver.find_elements(By.CSS_SELECTOR, 'div.row.gx-3 div.col-lg-8.col-12 div.mint-card')

for rto in rtos[0:1]:
    rto_url = rto.find_element(By.CSS_SELECTOR, 'div.card-inner div.card-copy a').get_attribute('href')
    base_url = rto_url.replace('summary', '')
    
    wb = openpyxl.Workbook() #Create a workbook

    driver.execute_script(f"window.open('{rto_url}', '_blank');")
    driver.switch_to.window(driver.window_handles[-1]) 
    print('Getting Data From: ',rto_url)
    
    # Scope Overview Tab - Download CSV
    scope_overview = base_url + '/scope_overview'
    driver.get(scope_overview)
    time.sleep(4)
    #print(driver.page_source)

    scope_ws = wb.active 
    scope_ws.title='Scope Overview'

    button_selector = '//*[contains(@class, "mt-4 mb-2 tableHeader")]/div'
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, button_selector))
    )
    result = driver.execute_script("""
        var xpath = arguments[0];
        var button = document.evaluate(xpath, document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
        if (button) {
            button.click();
            return 'Button clicked!';
        } else {
            return 'Button not found!';
        }
    """, button_selector)
    print(result)
    time.sleep(2)


    #button = driver.find_element(By.XPATH, button_selector)
    #driver.execute_script("arguments[0].click();", button)
    '''
    try:
        #dropdown_trigger_xpath = '//*[contains(@class, "mt-4 mb-2 tableHeader")]'
        #WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, dropdown_trigger_xpath)))

        button_selector = 'button#trigger-button_19'  # Example CSS selector

        # JavaScript to find the button and click it
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, button_selector)))

        result = driver.execute_script("""
            var button = document.querySelector(arguments[0]);
            if (button) {
                button.click();
                return 'Button clicked!';
            } else {
                return 'Button not found!';
            }
        """, button_selector)
        print(result)
        time.sleep(2)
        
        dropdown_trigger = driver.find_element(By.XPATH, dropdown_trigger_xpath)
        driver.execute_script("arguments[0].scrollIntoView(true);", dropdown_trigger)
        driver.execute_script("arguments[0].click();", dropdown_trigger)

        print('Seen and clicked')
        #print(dropdown_trigger_xpath)

        export_all_csv_button = driver.find_element(By.XPATH, '//button[contains(text(), "Export all as CSV")]')
        export_all_csv_button.click()

        time.sleep(5)
        downloaded_files = os.listdir(download_folder) # Check for downloaded CSV file
        csv_file = None
        for file in downloaded_files:
            if file.endswith('.csv'):
                csv_file = os.path.join(download_folder, file)
                break

        if csv_file:
            print(f"CSV file downloaded: {csv_file}")
            df = pd.read_csv(csv_file)
            #scope_ws = wb.create_sheet(title='scope_overview')

            for row in pd.DataFrame(df).itertuples(index=False, name=None):
                scope_ws.append(row)

            os.remove(csv_file)
        else:
            print("No CSV file found in the download folder.")

    except Exception as e:
        #scope_ws = wb.create_sheet(title='scope_overview')
        print("Error downloading CSV:", rto_url)'''
    

    # Save the workbook after populating both sheets
    wb.save(f"{rto_url.replace('https://training.gov.au/organisation/details/', '')}.xlsx")

    driver.close() #Close that tab
    driver.switch_to.window(driver.window_handles[0]) #switch back to the main tab
    time.sleep(1)
    #driver.quit()

Getting Data From:  https://training.gov.au/organisation/details/0022


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00ED33E3+25059]
	(No symbol) [0x00E5CDE4]
	(No symbol) [0x00D3BEC3]
	(No symbol) [0x00D7FD86]
	(No symbol) [0x00D7FFCB]
	(No symbol) [0x00DBD952]
	(No symbol) [0x00DA1F44]
	(No symbol) [0x00DBB51E]
	(No symbol) [0x00DA1C96]
	(No symbol) [0x00D73FAC]
	(No symbol) [0x00D74F3D]
	GetHandleVerifier [0x011C5543+3113795]
	GetHandleVerifier [0x011DA20A+3198986]
	GetHandleVerifier [0x011D29E2+3168226]
	GetHandleVerifier [0x00F73250+680016]
	(No symbol) [0x00E6572D]
	(No symbol) [0x00E629D8]
	(No symbol) [0x00E62B75]
	(No symbol) [0x00E557D0]
	BaseThreadInitThunk [0x7695FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77AE809E+286]
	RtlGetAppContainerNamedObjectPath [0x77AE806E+238]


In [37]:
driver.quit()

In [ ]:
#visit the addresses tab
    addresses =base_url + '/addresses'
    driver.get(addresses)
    addresses_sheet = wb.create_sheet('addresses')

    time.sleep(3)

    #visit the scope_overview tab
    scope_overview = base_url + '/scope_overview'
    driver.get(scope_overview)
    scope_sheet = wb.create_sheet('scope_overview')
    time.sleep(3)

    #visit the qualifications tab
    qualifications = base_url + '/qualifications'
    driver.get(qualifications)
    q_sheet = wb.create_sheet('qualifications')
    time.sleep(3)

    #visit the skill_sets tab
    skill_sets =base_url + '/skill_sets'
    driver.get(skill_sets)
    skills_sheet = wb.create_sheet('skill_sets')
    time.sleep(3)

    #visit the units tab
    units = base_url + '/units'
    driver.get(units)
    units_sheet = wb.create_sheet('units')
    time.sleep(3)

    #visit the courses tab
    courses = base_url + '/courses'
    driver.get(courses)
    courses_sheet = wb.create_sheet('courses')
    time.sleep(3)

In [1]:
#contact_name = entry.find_element(By.XPATH, './/table').text
        #job_title = entry.find_element(By.XPATH, './/h2').text
        #organisation_name = entry.find_element(By.XPATH, './/h2').text
        #phone = entry.find_element(By.XPATH, './/h2').text
        #fax = entry.find_element(By.XPATH, './/h2').text
        #email = entry.find_element(By.XPATH, './/h2').text
        #address = entry.find_element(By.XPATH, './/h2').text